In [14]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [15]:
more_features = False
use_staking = False
use_kernal_params = False
do_augment = True
fast = False

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [16]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [17]:
_, X_bottomhalf, _, y_bottomhalf = train_test_split(X, y, test_size=0.8, random_state=10)

In [18]:
if use_staking:
    X = X_bottomhalf
    y = y_bottomhalf

In [19]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [20]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

if use_kernal_params:
    # https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment (lb: 0.901)
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 13,
        "learning_rate" : 0.01,
        "bagging_freq": 5,
        "bagging_fraction" : 0.4,
        "feature_fraction" : 0.05,
        "min_data_in_leaf": 80,
        "min_sum_heassian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

In [21]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [22]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
predictions = test_df[['ID_code']]
val_aucs = []

In [24]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    N = 15
    
    if not do_augment:
        N = 1
    
    p_valid,yp = 0,0
    for i in range(N):
        if do_augment:
            X_t, y_t = augment(X_train.values, y_train.values)
            X_t = pd.DataFrame(X_t)
            X_t = X_t.add_prefix('var_')

            trn_data = lgb.Dataset(X_t, label=y_t)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        else:
            trn_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        
        evals_result = {}
        
        early_stopping_rounds = 800 if fast else 3000
        
        lgb_clf = lgb.train(params,
                        trn_data,
                        num_boost_round=2000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval = 300,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    
    oof['predict'][valid_index] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold_n+1)] = yp/N

Fold 0 started at Tue Mar 26 08:26:37 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.761029	valid_1's auc: 0.752524
[600]	training's auc: 0.800693	valid_1's auc: 0.793774
[900]	training's auc: 0.8219	valid_1's auc: 0.814963
[1200]	training's auc: 0.835716	valid_1's auc: 0.828576
[1500]	training's auc: 0.846276	valid_1's auc: 0.838533
[1800]	training's auc: 0.854118	valid_1's auc: 0.846108
[2100]	training's auc: 0.860732	valid_1's auc: 0.852308
[2400]	training's auc: 0.866213	valid_1's auc: 0.857418
[2700]	training's auc: 0.870876	valid_1's auc: 0.861756
[3000]	training's auc: 0.874628	valid_1's auc: 0.865214
[3300]	training's auc: 0.877906	valid_1's auc: 0.868199
[3600]	training's auc: 0.880904	valid_1's auc: 0.870886
[3900]	training's auc: 0.883463	valid_1's auc: 0.873234
[4200]	training's auc: 0.885889	valid_1's auc: 0.875354
[4500]	training's auc: 0.88791	valid_1's auc: 0.876998
[4800]	training's auc: 0.889854	valid_1's auc: 0.878736
[51

[900]	training's auc: 0.821151	valid_1's auc: 0.814527
[1200]	training's auc: 0.835213	valid_1's auc: 0.828424
[1500]	training's auc: 0.845788	valid_1's auc: 0.838603
[1800]	training's auc: 0.853616	valid_1's auc: 0.846051
[2100]	training's auc: 0.860124	valid_1's auc: 0.852007
[2400]	training's auc: 0.865575	valid_1's auc: 0.857105
[2700]	training's auc: 0.870084	valid_1's auc: 0.861447
[3000]	training's auc: 0.874107	valid_1's auc: 0.865286
[3300]	training's auc: 0.877422	valid_1's auc: 0.868274
[3600]	training's auc: 0.880326	valid_1's auc: 0.871044
[3900]	training's auc: 0.882875	valid_1's auc: 0.873369
[4200]	training's auc: 0.885234	valid_1's auc: 0.875324
[4500]	training's auc: 0.887185	valid_1's auc: 0.876963
[4800]	training's auc: 0.888985	valid_1's auc: 0.878614
[5100]	training's auc: 0.890635	valid_1's auc: 0.880142
[5400]	training's auc: 0.892149	valid_1's auc: 0.881539
[5700]	training's auc: 0.893578	valid_1's auc: 0.882732
[6000]	training's auc: 0.894896	valid_1's auc: 0.

[9000]	training's auc: 0.903574	valid_1's auc: 0.890818
[9300]	training's auc: 0.904187	valid_1's auc: 0.891294
[9600]	training's auc: 0.90475	valid_1's auc: 0.891817
[9900]	training's auc: 0.905279	valid_1's auc: 0.892223
[10200]	training's auc: 0.905774	valid_1's auc: 0.892643
[10500]	training's auc: 0.906261	valid_1's auc: 0.893008
[10800]	training's auc: 0.906722	valid_1's auc: 0.893415
[11100]	training's auc: 0.907151	valid_1's auc: 0.893778
[11400]	training's auc: 0.907519	valid_1's auc: 0.894085
[11700]	training's auc: 0.907911	valid_1's auc: 0.894424
[12000]	training's auc: 0.908283	valid_1's auc: 0.894756
[12300]	training's auc: 0.908646	valid_1's auc: 0.895057
[12600]	training's auc: 0.90899	valid_1's auc: 0.895294
[12900]	training's auc: 0.909308	valid_1's auc: 0.895554
[13200]	training's auc: 0.909633	valid_1's auc: 0.895825
[13500]	training's auc: 0.909894	valid_1's auc: 0.896026
[13800]	training's auc: 0.910193	valid_1's auc: 0.896285
[14100]	training's auc: 0.910449	vali

[12600]	training's auc: 0.909108	valid_1's auc: 0.89574
[12900]	training's auc: 0.90944	valid_1's auc: 0.896068
[13200]	training's auc: 0.909739	valid_1's auc: 0.896269
[13500]	training's auc: 0.910019	valid_1's auc: 0.896497
[13800]	training's auc: 0.910313	valid_1's auc: 0.89673
[14100]	training's auc: 0.910555	valid_1's auc: 0.896935
[14400]	training's auc: 0.910826	valid_1's auc: 0.897139
[14700]	training's auc: 0.911051	valid_1's auc: 0.89735
[15000]	training's auc: 0.911293	valid_1's auc: 0.89756
[15300]	training's auc: 0.911503	valid_1's auc: 0.89771
[15600]	training's auc: 0.91169	valid_1's auc: 0.897852
[15900]	training's auc: 0.911901	valid_1's auc: 0.898014
[16200]	training's auc: 0.912084	valid_1's auc: 0.898146
[16500]	training's auc: 0.912275	valid_1's auc: 0.898312
[16800]	training's auc: 0.912463	valid_1's auc: 0.898456
[17100]	training's auc: 0.912661	valid_1's auc: 0.898607
[17400]	training's auc: 0.912847	valid_1's auc: 0.898719
[17700]	training's auc: 0.91302	valid_

[15900]	training's auc: 0.912155	valid_1's auc: 0.898012
[16200]	training's auc: 0.912345	valid_1's auc: 0.898132
[16500]	training's auc: 0.912523	valid_1's auc: 0.898317
[16800]	training's auc: 0.912726	valid_1's auc: 0.89845
[17100]	training's auc: 0.912897	valid_1's auc: 0.898503
[17400]	training's auc: 0.913087	valid_1's auc: 0.898634
[17700]	training's auc: 0.913243	valid_1's auc: 0.898713
[18000]	training's auc: 0.913393	valid_1's auc: 0.898781
[18300]	training's auc: 0.913548	valid_1's auc: 0.898891
[18600]	training's auc: 0.913697	valid_1's auc: 0.89899
[18900]	training's auc: 0.913856	valid_1's auc: 0.899106
[19200]	training's auc: 0.913997	valid_1's auc: 0.899181
[19500]	training's auc: 0.914145	valid_1's auc: 0.89927
[19800]	training's auc: 0.914281	valid_1's auc: 0.899395
[20100]	training's auc: 0.914436	valid_1's auc: 0.899464
[20400]	training's auc: 0.914571	valid_1's auc: 0.899556
[20700]	training's auc: 0.914679	valid_1's auc: 0.899618
[21000]	training's auc: 0.914807	v

...

[4800]	training's auc: 0.88961	valid_1's auc: 0.874047
[5100]	training's auc: 0.891189	valid_1's auc: 0.875629
[5400]	training's auc: 0.892661	valid_1's auc: 0.877087
[5700]	training's auc: 0.894059	valid_1's auc: 0.878391
[6000]	training's auc: 0.895282	valid_1's auc: 0.879592
[6300]	training's auc: 0.896481	valid_1's auc: 0.880697
[6600]	training's auc: 0.897597	valid_1's auc: 0.881847
[6900]	training's auc: 0.898588	valid_1's auc: 0.882759
[7200]	training's auc: 0.899554	valid_1's auc: 0.883677
[7500]	training's auc: 0.900438	valid_1's auc: 0.884478
[7800]	training's auc: 0.901244	valid_1's auc: 0.885297
[8100]	training's auc: 0.901999	valid_1's auc: 0.885943
[8400]	training's auc: 0.90269	valid_1's auc: 0.886605
[8700]	training's auc: 0.903402	valid_1's auc: 0.887321
[9000]	training's auc: 0.904013	valid_1's auc: 0.887849
[9300]	training's auc: 0.904589	valid_1's auc: 0.888346
[9600]	training's auc: 0.905099	valid_1's auc: 0.888805
[9900]	training's auc: 0.905636	valid_1's auc: 0.8

[14700]	training's auc: 0.911203	valid_1's auc: 0.893869
[15000]	training's auc: 0.911427	valid_1's auc: 0.894013
[15300]	training's auc: 0.911656	valid_1's auc: 0.894226
[15600]	training's auc: 0.911886	valid_1's auc: 0.894389
[15900]	training's auc: 0.912103	valid_1's auc: 0.894616
[16200]	training's auc: 0.912298	valid_1's auc: 0.89473
[16500]	training's auc: 0.912499	valid_1's auc: 0.894911
[16800]	training's auc: 0.912676	valid_1's auc: 0.895066
[17100]	training's auc: 0.912856	valid_1's auc: 0.895168
[17400]	training's auc: 0.913034	valid_1's auc: 0.8953
[17700]	training's auc: 0.913215	valid_1's auc: 0.8954
[18000]	training's auc: 0.913367	valid_1's auc: 0.8955
[18300]	training's auc: 0.913544	valid_1's auc: 0.895609
[18600]	training's auc: 0.91369	valid_1's auc: 0.895718
[18900]	training's auc: 0.913826	valid_1's auc: 0.895756
[19200]	training's auc: 0.913982	valid_1's auc: 0.895854
[19500]	training's auc: 0.91412	valid_1's auc: 0.895934
[19800]	training's auc: 0.914253	valid_1

[23700]	training's auc: 0.915359	valid_1's auc: 0.896768
[24000]	training's auc: 0.915478	valid_1's auc: 0.896807
[24300]	training's auc: 0.915573	valid_1's auc: 0.896825
[24600]	training's auc: 0.915669	valid_1's auc: 0.896853
[24900]	training's auc: 0.915772	valid_1's auc: 0.896899
[25200]	training's auc: 0.915871	valid_1's auc: 0.896936
[25500]	training's auc: 0.915968	valid_1's auc: 0.896965
[25800]	training's auc: 0.916067	valid_1's auc: 0.896977
[26100]	training's auc: 0.916177	valid_1's auc: 0.897047
[26400]	training's auc: 0.916274	valid_1's auc: 0.897085
[26700]	training's auc: 0.916378	valid_1's auc: 0.897085
[27000]	training's auc: 0.916491	valid_1's auc: 0.897109
[27300]	training's auc: 0.916579	valid_1's auc: 0.897133
[27600]	training's auc: 0.916676	valid_1's auc: 0.89716
[27900]	training's auc: 0.916777	valid_1's auc: 0.89716
[28200]	training's auc: 0.916877	valid_1's auc: 0.897154
[28500]	training's auc: 0.916989	valid_1's auc: 0.897175
[28800]	training's auc: 0.917091	

[33900]	training's auc: 0.919236	valid_1's auc: 0.89712
[34200]	training's auc: 0.919336	valid_1's auc: 0.897105
[34500]	training's auc: 0.919438	valid_1's auc: 0.897091
Early stopping, best iteration is:
[31505]	training's auc: 0.918477	valid_1's auc: 0.89721
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.763415	valid_1's auc: 0.760105
[600]	training's auc: 0.802283	valid_1's auc: 0.795257
[900]	training's auc: 0.823741	valid_1's auc: 0.814087
[1200]	training's auc: 0.837166	valid_1's auc: 0.825902
[1500]	training's auc: 0.847581	valid_1's auc: 0.835467
[1800]	training's auc: 0.855622	valid_1's auc: 0.842371
[2100]	training's auc: 0.862113	valid_1's auc: 0.848227
[2400]	training's auc: 0.867272	valid_1's auc: 0.852912
[2700]	training's auc: 0.871899	valid_1's auc: 0.857124
[3000]	training's auc: 0.875645	valid_1's auc: 0.86058
[3300]	training's auc: 0.879076	valid_1's auc: 0.863623
[3600]	training's auc: 0.881919	valid_1's auc: 0.866126
[3900]	

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

In [25]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.901479053, std: 0.003126603. All auc: 0.901466518.


In [26]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
